# Bedrock Knowledge Base Retrieval and Generation with SageMaker Inference and Reranking

## 1: Import and Load Variables

In [ ]:
import json

# Load the configuration variables from a JSON file
with open("../Lab 1/variables.json", "r") as f:
    variables = json.load(f)

variables


## 2: Define ARN and Configuration Details

In [ ]:
# Knowledge Base Selection  
kb_id = variables["kbFixedChunk"]  # Options: "kbFixedChunk", "kbHierarchicalChunk", "kbSemanticChunk"

# Retrieval-Augmented Generation (RAG) Configuration  
number_of_results = 3  # Number of relevant documents to retrieve  
generation_configuration = {
    "temperature": 0,  # Lower temperature for more deterministic responses  
    "top_k": 10,  # Consider top 10 tokens at each generation step  
    "max_new_tokens": 5000,  # Maximum number of tokens to generate  
    "stop": "<|eot_id|>"  # Stop sequence to end the response generation  
}

# Define ARNs (Amazon Resource Names) for the model
rerank_model_arn=f"arn:aws:bedrock:us-west-2::foundation-model/cohere.rerank-v3-5:0"

# User Query
query = "what was the % increase in sales?"  # Sample query to retrieve data from the knowledge base


## 3: Set Up Bedrock Client

In [ ]:
import boto3
import json
from typing import *

# Configure the Bedrock client
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name="us-west-2")


## 4: Define Function for Reranking

In [ ]:
import boto3

# Initialize Bedrock client to interact with the Bedrock Knowledge Base
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime", region_name=variables["regionName"])
bedrock_client = boto3.client("bedrock-runtime", region_name=variables["regionName"])

# Constants for Knowledge Base ID, SageMaker endpoint, and number of results to retrieve
KNOWLEDGE_BASE_ID = kb_id
ENDPOINT_NAME = variables['sagemakerLLMEndpoint']
NUM_RESULTS = number_of_results

def search_knowledge_base(query, region_name, kb_id, num_results=5, use_reranking=False, model_arn=None):
    """
    Search Bedrock Knowledge Base and optionally rerank results.
    
    Parameters:
        query (str): Search query text
        region_name (str): AWS region name (e.g. "us-east-1")
        kb_id (str): Knowledge Base ID
        num_results (int): Maximum number of results to return
        use_reranking (bool): Whether to rerank results for better relevance
        model_arn (str): Model ARN for reranking (required if use_reranking=True)
    
    Returns:
        list: Retrieved document texts
    """
    # Create client for Bedrock
    client = boto3.client("bedrock-agent-runtime", region_name=region_name)
    
    # Step 1: Search the knowledge base
    try:
        # Call the retrieve API
        response = client.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={"text": query},
            retrievalConfiguration={
                "vectorSearchConfiguration": {"numberOfResults": num_results}
            }
        )
        
        # Get document texts from results
        documents = [result['content']['text'] for result in response['retrievalResults']]
        print(f"Found {len(documents)} documents")
        
    except Exception as e:
        print(f"Search failed: {e}")
        return []
    
    # Step 2: Rerank results if requested
    if use_reranking and model_arn and documents:
        try:
            # Format documents for reranking API
            sources = [
                {
                    "inlineDocumentSource": {"textDocument": {"text": doc}, "type": "TEXT"},
                    "type": "INLINE"
                } 
                for doc in documents
            ]
            
            # Call rerank API
            rerank_response = client.rerank(
                queries=[{"textQuery": {"text": query}, "type": "TEXT"}],
                rerankingConfiguration={
                    "bedrockRerankingConfiguration": {
                        "modelConfiguration": {"modelArn": model_arn},
                        "numberOfResults": num_results
                    },
                    "type": "BEDROCK_RERANKING_MODEL"
                },
                sources=sources
            )
            
            # Get reordered documents
            reranked_docs = []
            for result in rerank_response.get("results", []):
                idx = result.get("index", 0)
                if 0 <= idx < len(documents):
                    reranked_docs.append(documents[idx])
            
            print(f"Reranked results: {len(reranked_docs)} documents")
            return reranked_docs
                
        except Exception as e:
            print(f"Reranking failed: {e}")
            print("Using original search results instead")
    
    # Return the documents
    return documents

## 5. Define SageMaker & Bedrock helper functions

In [ ]:
# Function to format the prompt for Llama 3 model using retrieved context
def format_prompt(query, context):
    """Format prompt for Llama 3"""
    # Create the system prompt that includes the context and the user's question
    system_prompt = f"""Use the following context to answer the question. If you don't know the answer, say 'I don't know'.
        Context:
        {" ".join(context)}"
    """

    # Format the complete prompt including system and user instructions
    return f"""
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|start_header_id|>user<|end_header_id|>
        Question: {query}
        <|start_header_id|>assistant<|end_header_id|>
        """.strip()

# Function to generate a response from the SageMaker endpoint based on the formatted prompt
def generate_response(prompt):
    """Generate response using SageMaker endpoint"""
    # Initialize SageMaker runtime client
    runtime = boto3.client('sagemaker-runtime')
    
    # Prepare the payload with prompt and generation parameters
    payload = {
        "inputs": prompt,  # The formatted prompt to pass to the model
        "parameters": generation_configuration  # Additional parameters for the model (e.g., temperature, tokens)
    }
    try:
        # Call the SageMaker endpoint to generate the response
        response = runtime.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,  # SageMaker endpoint name
            ContentType='application/json',  # Content type for the request
            Body=json.dumps(payload)  # Send the payload as JSON
        )

        # Parse the response body
        result = json.loads(response['Body'].read().decode("utf-8"))
        
        # Handle different response formats (list or dictionary)
        if isinstance(result, list):
            # If the result is a list, extract the generated text from the first element
            return result[0]['generated_text']
        elif 'generated_text' in result:
            # If the result is a dictionary with 'generated_text', return the generated text
            return result['generated_text']
        elif 'generation' in result:
            # Alternative format with 'generation' key
            return result['generation']
        else:
            # Raise an error if the response format is unexpected
            raise RuntimeError("Unexpected response format")
            
    except Exception as e:
        # Raise an error if the generation process fails
        raise RuntimeError(f"Generation failed: {str(e)}")

## 6: Compare the Retrieved results WITH & WITHOUT Reranking

In [ ]:
# Example usage
query = "Compare the results between 2022 and 2023"

print("WITHOUT RERANKING:")
# Retrieve relevant context without reranking
context_without_reranking = search_kb_with_optional_rerank(query, use_reranking=False)


In [ ]:
print("\nWITH RERANKING:")
# Retrieve relevant context with reranking
context_with_reranking = search_kb_with_optional_rerank(
    query=query, 
    model_arn=rerank_model_arn, 
    use_reranking=True
)


## 7: Compare the Generated results WITH & WITHOUT Reranking

In [ ]:
print("WITHOUT RERANKING:")

# Format the prompt by combining the user's query and the retrieved context
prompt_without_reranking = format_prompt(query, context_without_reranking)

# Generate the response using the formatted prompt by calling the SageMaker endpoint
response_without_reranking = generate_response(prompt_without_reranking)

# Print the user's query and answer
print("Question:", query)
print("Answer:", response_without_reranking)


In [ ]:
print("WITH RERANKING:")

# Format the prompt by combining the user's query and the retrieved context
prompt_with_reranking = format_prompt(query, context_with_reranking)

# Generate the response using the formatted prompt by calling the SageMaker endpoint
response_with_reranking = generate_response(prompt_with_reranking)

# Print the user's query and answer
print("Question:", query)
print("Answer:", response_with_reranking)